In [0]:
-- =========================================================
-- GOLD: ML - feature_ml_price
-- Objetivo: predecir price (target_price)
-- Fuentes: airbnb.silver.listings, airbnb.silver.hosts, airbnb.silver.listing_amenities
-- =========================================================

-- 1) Asegurar esquema Gold
CREATE SCHEMA IF NOT EXISTS airbnb.gold;

-- 2) Construcción de features + target
CREATE OR REPLACE TABLE airbnb.gold.feature_ml_price AS
WITH amen AS (
  SELECT
    bk_listing_id,
    COUNT(*) AS num_amenities
  FROM airbnb.silver.listing_amenities
  GROUP BY bk_listing_id
)
SELECT
  /* Identificador estable por clave de negocio */
  l.bk_listing_id                                   AS listing_id,

  -- ===== TARGET =====
  TRY_CAST(l.price AS DECIMAL(10,2))                AS target_price,

  -- ===== FEATURES NUMÉRICAS =====
  TRY_CAST(l.accommodates AS INT)                   AS accommodates,
  TRY_CAST(l.bedrooms AS INT)                       AS bedrooms,
  TRY_CAST(l.beds AS INT)                           AS beds,
  TRY_CAST(l.bathrooms AS DECIMAL(3,1))             AS bathrooms,
  TRY_CAST(l.minimum_nights AS INT)                 AS minimum_nights,
  TRY_CAST(l.maximum_nights AS INT)                 AS maximum_nights,
  TRY_CAST(l.number_of_reviews AS INT)              AS number_of_reviews,
  TRY_CAST(l.review_scores_rating AS DECIMAL(5,2))  AS review_scores_rating,
  COALESCE(a.num_amenities, 0)                      AS num_amenities,

  -- ===== FEATURES CATEGÓRICAS / BOOLEANAS =====
  CASE
    WHEN CAST(h.host_is_superhost AS STRING) IN ('true','True','1')  THEN TRUE
    WHEN CAST(h.host_is_superhost AS STRING) IN ('false','False','0') THEN FALSE
    ELSE TRY_CAST(h.host_is_superhost AS BOOLEAN)
  END                                               AS host_is_superhost,

  l.property_type,
  l.room_type,
  l.country

FROM airbnb.silver.listings l
LEFT JOIN airbnb.silver.hosts h
  ON h.host_sk = l.host_sk
LEFT JOIN amen a
  ON a.bk_listing_id = l.bk_listing_id

-- 3) Filtros mínimos de calidad
WHERE l.price IS NOT NULL
  AND TRY_CAST(l.price AS DECIMAL(10,2)) > 0;
